In [15]:
import google.generativeai as genai

genai.configure(api_key="<your key>")


def get_completion(prompt):
    """
    Get the completion for a given prompt using the specified model.
    Returns the answer with the highest score.
    """
    model = genai.GenerativeModel(
            "models/gemini-1.5-flash",
            system_instruction="You are a user.",
        )
    response = model.generate_content(prompt)
    return response.text


# Strategy: “Chain-of-Thought Prompting” (CoT)

Guiding the model through a step-by-step reasoning process. Instead of prompting the model to directly provide an answer

Give the model "time to think"

## Tactic: Specify the steps required to complete a task

In [12]:
text = f"""
In a charming village, siblings Jack and Jill set out on 
a quest to fetch water from a hilltop 
well. As they climbed, singing joyfully, misfortune 
struck—Jack tripped on a stone and tumbled 
down the hill, with Jill following suit. 
Though slightly battered, the pair returned home to 
comforting embraces. Despite the mishap, 
their adventurous spirits remained undimmed, and they 
continued exploring with delight.
"""

# example 1
prompt = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""

print(get_completion(prompt))

Jack and Jill went up a hill to fetch water but unfortunately tumbled down.

Jack et Jill sont montés sur une colline pour aller chercher de l'eau, mais malheureusement, ils sont tombés.

Jack, Jill

```json
{
  "french_summary": "Jack et Jill sont montés sur une colline pour aller chercher de l'eau, mais malheureusement, ils sont tombés.",
  "num_names": 2
}
```



## Tactic: Ask for output in a specified format

In [13]:
prompt = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""

print(get_completion(prompt))

Text: <
In a charming village, siblings Jack and Jill set out on 
a quest to fetch water from a hilltop 
well. As they climbed, singing joyfully, misfortune 
struck—Jack tripped on a stone and tumbled 
down the hill, with Jill following suit. 
Though slightly battered, the pair returned home to 
comforting embraces. Despite the mishap, 
their adventurous spirits remained undimmed, and they 
continued exploring with delight.
>
Summary: Jack and Jill went up a hill to fetch water but unfortunately tumbled down.
Translation: Jack et Jill sont montés sur une colline pour chercher de l'eau, mais malheureusement, ils sont tombés.
Names: Jack, Jill
Output JSON: {"french_summary": "Jack et Jill sont montés sur une colline pour chercher de l'eau, mais malheureusement, ils sont tombés.", "num_names": 2}



## Tactic: Instruct the model to work out its own solution before rushing to a conclusion

In [16]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
A bakery sells two types of pastries: croissants and muffins. 
Croissants cost $3 each, and muffins cost $2 each. 
Yesterday, the bakery sold a total of 50 pastries and made $130 in revenue. 
How many croissants and muffins did the bakery sell?

Student's Solution:
Let x be the number of croissants the baker sold, and y be the number of muffins he sold.
than x+y=50
3x+2y=130
So, the bakery sold 20 croissants and 30 muffins
"""

print(get_completion(prompt))

The student's solution is correct.  Let's check:

* **Equation 1:** x + y = 50 (Total pastries)
* **Equation 2:** 3x + 2y = 130 (Total revenue)

If x = 20 (croissants) and y = 30 (muffins):

* Equation 1: 20 + 30 = 50 (Correct)
* Equation 2: (3 * 20) + (2 * 30) = 60 + 60 = 120 (Incorrect)

There's a small arithmetic error in the student's revenue calculation. While the setup of the equations is correct, the solution (20 croissants, 30 muffins) does not satisfy the revenue equation.  Therefore, the **student's solution is incorrect**, even though the approach is right.  They need to solve the system of equations properly to find the correct number of croissants and muffins.



***Note that the student's solution is actually not correct.***

***We can fix this by instructing the model to work out its own solution first.***

In [17]:
prompt = f"""
Your task is to determine if the student's solution 
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total. 
- Then compare your solution to the student's solution 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution 
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
A bakery sells two types of pastries: croissants and muffins. 
Croissants cost $3 each, and muffins cost $2 each. 
Yesterday, the bakery sold a total of 50 pastries and made $130 in revenue. 
How many croissants and muffins did the bakery sell?
``` 
Student's solution:
```
Let x be the number of croissants the baker sold, and y be the number of muffins he sold.
than x+y=50
3x+2y=130
So, the bakery sold 20 croissants and 30 muffins
```
Actual solution:
"""

print(get_completion(prompt))

Actual solution:
```
Let x be the number of croissants and y be the number of muffins.
We can set up a system of two equations based on the given information:
x + y = 50  (total number of pastries)
3x + 2y = 130 (total revenue)

We can solve this system of equations using substitution or elimination. Let's use elimination:
Multiply the first equation by -2:
-2x - 2y = -100
Add this to the second equation:
3x + 2y = 130
-2x - 2y = -100
----------------
x = 30

Now substitute x = 30 into the first equation:
30 + y = 50
y = 50 - 30
y = 20

Therefore, the bakery sold 30 croissants and 20 muffins.
```
Is the student's solution the same as actual solution just calculated:
```
no
```
Student grade:
```
incorrect
```



# Strategy: Avoid Model Hallucinations
- Boie is a real company, the product name is not real.

In [18]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""

print(get_completion(prompt))

The Boie AeroGlide UltraSlim Smart Toothbrush is a relatively new entrant in the electric toothbrush market, distinguishing itself primarily through its design and sustainability focus.  Information available online suggests the following key features:

* **Ultra-slim design:** As the name implies, it's designed to be exceptionally thin and lightweight, making it more comfortable to hold and maneuver in the mouth, especially for those with smaller mouths or sensitive gums.

* **Smart features (limited):**  While marketed as "smart," its smart features are less extensive than high-end smart toothbrushes.  It likely refers to features such as a built-in timer to ensure you brush for the recommended two minutes.  It may lack features like pressure sensors, app connectivity, or multiple brushing modes found in more sophisticated models.

* **Replaceable brush head:**  This is a crucial aspect for sustainability. Instead of discarding the entire toothbrush, users replace only the brush head

## Tactic: Give the model option to say it doesn't know the answer to a question			

In [19]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie. Only answer if you know the answer with certainty.
"""

print(get_completion(prompt))

I do not have access to real-time information, including specific product details like those for the AeroGlide UltraSlim Smart Toothbrush by Boie.  My knowledge is based on the data I was trained on, which is not a live, updated product catalog.  To get accurate information on this toothbrush, you should check the Boie website or a reputable retailer that sells their products.



## Tactic: Ask the model to find evidence before answering						

In [77]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie. Find evidence before answering in https://boieusa.com.  
"""

print(get_completion(prompt))

Based on the provided link (boieusa.com), I cannot find a toothbrush called "AeroGlide UltraSlim Smart Toothbrush"  in Boie's product line.  Their website showcases several toothbrush models, but none match that specific name.  Therefore, I cannot provide information about it.

